In [1]:
#util
import os
import time

#Math/arrays
import numpy as np
import pandas as pd

import optimizationFuncs as optim
import cpso
from importlib import reload

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 

#File io
#from Bio.PDB import *
from loadFile import *
from makePDB import *

#PSO
from threeDMaxGrad import *
from myUtils import *
# Import PySwarms
import pyswarms as ps
import numpy.random as random

%load_ext autoreload
%autoreload 2


from importlib import reload

In [2]:
#==========================================================================
# Make directory if it doesn't exist
#--------------------------------------------------------------------------
if not os.path.exists('my_folder'):
    os.makedirs('my_folder')

In [3]:
## constants
#inFile = "../../examples/GSDB/inputs/KR_1mb/chr20_matrix.txt"
inFile = "../../examples/GSDB/inputs/KR_1mb/chr20_list.txt"
#inFile = "../../examples/input/chr19.txt"

## load file 
lstCons=loadFile.loadFileFunc(loadFile(), inFile)
 

this is a sparse matrix
Number of points :  1891
numBins  62.0
[[0.00000000e+00 1.00000000e+00 8.83912865e+04]
 [0.00000000e+00 5.50000000e+01 2.24361671e+03]
 [0.00000000e+00 2.50000000e+01 3.61152985e+03]
 ...
 [6.00000000e+01 6.20000000e+01 1.42017020e+04]
 [6.00000000e+01 6.10000000e+01 6.48508182e+04]
 [6.10000000e+01 6.20000000e+01 9.34392380e+04]]


In [4]:
lstCons[:,0] = lstCons[:,0].astype('int')
lstCons[:,1] = lstCons[:,1].astype('int')

scaleFactor = 25/max(lstCons[:,2])
lstCons[:,2] = lstCons[:,2]*scaleFactor
lstCons

array([[ 0.        ,  1.        , 16.61958593],
       [ 0.        , 55.        ,  0.42185132],
       [ 0.        , 25.        ,  0.67905031],
       ...,
       [60.        , 62.        ,  2.67024517],
       [60.        , 61.        , 12.19343883],
       [61.        , 62.        , 17.56871639]])

In [5]:
class outputObj:
    def __init__(self, xyzData, outputFile):
        self.xyzData = xyzData
        self.outputFile = outputFile
        
        #defaults
        self.recordName = []
        for i in range(len(xyzData)):
            self.recordName.append('ATOM')
            

    def myfunc(self):
        print("Hello my name is David")

In [6]:
def cost(xOyOz):
    cost = 0
    d = 0;
    structure = variables;
    

    #loop through existing data
    for k in range(len(lstCons)):
        print("yo2", lstCons)
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        
        if (IF <= 0) :
            continue
        

        if (abs(i - j) == 1 ) :
            IF = 1.0 * maxIF

        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

        z = str_dist - dist;

        # objective function       
        d += (z**2);     

        print(d)

        # cost
        cost = -(n/2) - (n*np.log(np.sqrt(d/n)));

        return cost

In [7]:
def getSpear():
    SUM = 0.0;

    Len = int(n * (n - 1) / 2) 
    Dist = np.zeros(Len);
    WishDist = np.zeros(Len);
    count = 1;
    structure = variables;
    for k in range(len(lstCons)):
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        # structure distance   
        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );
        SUM = SUM + ((str_dist - dist)**2);

        # calculate spearman_correlation and Pearson correlation
        if (i != j and IF > 0 and count<=Len  ):   
            Dist[count] = str_dist;
            WishDist[count]= dist;
            count = count + 1

    SUM = SUM / len(lstCons);    
    rmse = np.sqrt(SUM);

    # let's convert to a dataframe
    df = pd.DataFrame({'Dist': Dist, 'wishDist': WishDist})
    pearsonCoeff = df.corr(method = 'pearson')
    spearmanCoeff = df.corr(method = 'spearman')
    print("preoptim  : ", spearmanCoeff.iloc[0]['wishDist']) 

In [8]:
def opt_func(X):
    n_particles = X.shape[0]  # number of particles
    
    d = 0;
    #print(X)


    structure = variables #this is xyz
    allDists = np.zeros(n_particles)
    
    cost= 0
    for particle in range(n_particles):
        structure[:,changedXYZ] =  X[particle]
        
        #loop through existing data
        for k in range(len(lstCons)):
            i = int(lstCons[k,0]);    
            j = int(lstCons[k,1]);    
            IF = lstCons[k,2];  
            dist = lstCons[k,3];
            if (IF <= 0) :
                continue


            if (abs(i - j) == 1 ) :
                IF = 1.0 * maxIF

            x1=structure[i][0];  x2=structure[j][0];
            y1=structure[i][1];  y2=structure[j][1];
            z1=structure[i][2];  z2=structure[j][2];

            str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

            z = str_dist - dist;

            # objective function  
            d += (z**2)
            
            #cost
            cost += -(n/2) - (n*np.log(np.sqrt(d/n)));
            
        
        allDists[particle] = cost
    
    return allDists

In [12]:
## this is convert to distance
#CONVERT_FACTOR_R = np.arange(0.1, 2, .2) # this is alpha and can be looped
CONVERT_FACTOR_R=[0.9]
MAX_ITERATION = 500; # maximum number of iterations

AVG_DIST = 10.0  # an arbitrary distance

n=int(max(max(lstCons[:,0]),max(lstCons[:,1])))+1
print("n = ",n)

bestAlpha= CONVERT_FACTOR_R[0]
lstConsReset = lstCons
for CONVERT_FACTOR in CONVERT_FACTOR_R : 
    lstCons = lstConsReset
    ## Find the average IF
    avgIF = 0.0
    for i in range(len(lstCons)):
        avgIF = avgIF + float(lstCons[i][2])
    avgIF = avgIF/len(lstCons)
    
    maxIF = 0.0
    ## scale average distance to AVG_DIST
    avgDist = 10.0;
    avgAdjIF = 0.0;
    avgAdjCount = 0;
    totalIF = 0;

    for i in range(len(lstCons)):
        x = lstCons[i][0]
        y = lstCons[i][1]
        IF = lstCons[i][2]
        lstCons[i][2] = IF / avgIF  # normallize IF by avgIF
        IF = lstCons[i][2]
        dist = 1/(IF**CONVERT_FACTOR)
        avgDist = avgDist + dist

        totalIF = totalIF +  IF

        if ( IF > maxIF):
            maxIF = lstCons[i][2]
        # Find the adjacent position IF
        if ( abs(x-y)==1):
            avgAdjCount= avgAdjCount+1
            avgAdjIF =  avgAdjIF + IF

    avgDist = avgDist/len(lstCons)
    avgAdjIF = avgAdjIF/avgAdjCount

    maxIF = min(avgAdjIF, maxIF)

    ## TODO Add adjacent if none exist
    
    print('TODO Added missing adjacent constraint...')

    print('Number of constraints: = ', n)
    maxD = 0
    distResultsList= []
    for i in range(len(lstCons)):
        IF = lstCons[i,2];
        dist = AVG_DIST/ ((IF**CONVERT_FACTOR)* avgDist)
        distResultsList.append(dist)
        if (dist > maxD):
            maxD = dist;

    result = np.hstack((lstCons, np.atleast_2d(distResultsList).T))
    lstCons = result
    print('Max distance is: = \n', maxD); 
    
    ## Optimization
    ## Initialize Structure
    #=========================================================================
    
    smooth_factor = 1e-6 ; # for numerical stability
    LEARNING_RATE = 0.1; # Specify the learning rate
    NEAR_ZERO =0.00001; # used to signify a boundary of convergence
    NUM_FIGURES_OUTPUT=5

    thisStr =  [];
    R = [-0.5,0.5];
    for i in range(n):
        xyz = np.array([random.random(),random.random(),random.random()]) * (R[1]-R[0]) + min(R) 
        thisStr.append(xyz)
    
    ## Variables declaration
    #=========================================================================
    thislen = len(lstCons)

    Sum_Grad = np.zeros(n);
    variables = thisStr;
    oldobj = 0;
    getSpear()
    ######### PSO
    variables = np.array(variables)

    dist = 1/(IF**CONVERT_FACTOR)

    swarm_size = 20
    dim = n       # Dimension of X
    epsilon = 1
    options = {'c1': 0.5, 'c2':0.3, 'w':0.9}

    # Call an instance of PSO
    #optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
    #                                dimensions=dim,
    #                                options=options)
    
    options = {'c1': 0.5, 'c2':0.3, 'w':0.9, 'k': 3, 'p': 2}
    optimizer = ps.single.LocalBestPSO(n_particles=swarm_size,
                                    dimensions=dim,
                                    options=options)

    # Perform optimization
    for i in range(7):
        changedXYZ = np.random.randint(3)
        cost, bestPXYZ = optimizer.optimize(opt_func, iters=500)   
        variables[:,changedXYZ] =  bestPXYZ
        
        getSpear()
        print("cost = ", cost)
        if i == 0:
            lastCost = cost
            continue
        if cost/lastCost < 0.003:
            break
        
        
    

    #========================================================================
    # scoring using spearman correlation, pearson correlation and  RMSD     
    #------------------------------------------------------------------------
    # calculate rmse    
    SUM = 0.0;

    Len = int(n * (n - 1) / 2) 
    Dist = np.zeros(Len);
    WishDist = np.zeros(Len);
    count = 1;
    structure = variables;
    for k in range(len(lstCons)):
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        # structure distance   
        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );
        SUM = SUM + ((str_dist - dist)**2);

        # calculate spearman_correlation and Pearson correlation
        if (i != j and IF > 0 and count<=Len  ):   
            Dist[count] = str_dist;
            WishDist[count]= dist;
            count = count + 1

    SUM = SUM / len(lstCons);    
    rmse = np.sqrt(SUM);

    # let's convert to a dataframe
    df = pd.DataFrame({'Dist': Dist, 'wishDist': WishDist})
    pearsonCoeff = df.corr(method = 'pearson')
    spearmanCoeff = df.corr(method = 'spearman')
   
    if CONVERT_FACTOR == CONVERT_FACTOR_R[0]:#first run
        bestSpearmanRHO = spearmanCoeff.iloc[0]['wishDist']
        bestPearsonRHO = pearsonCoeff.iloc[0]['wishDist']
        bestMat =  variables
    elif bestSpearmanRHO < spearmanCoeff.iloc[0]['wishDist']:
        bestSpearmanRHO = spearmanCoeff.iloc[0]['wishDist']
        bestPearsonRHO = pearsonCoeff.iloc[0]['wishDist']
        bestMat =  variables
        bestAlpha = CONVERT_FACTOR

print("RMSE  : ", rmse)    
print("bestPearsonRHO  : ", bestPearsonRHO) 
print("bestSpearmanRHO  : ", bestSpearmanRHO) 
print("Best Alpha : ",bestAlpha)

2020-11-22 21:57:40,634 - pyswarms.single.local_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 3, 'p': 2}
pyswarms.single.local_best:   0%|          |0/500

n =  63
TODO Added missing adjacent constraint...
Number of constraints: =  63
Max distance is: = 
 66.69620869552902
preoptim  :  0.16672207476003062


pyswarms.single.local_best: 100%|██████████|500/500, best_cost=-1.32e+7
2020-11-22 22:00:14,019 - pyswarms.single.local_best - INFO - Optimization finished | best cost: -13195033.800299179, best pos: [ 0.51538832  0.11716764  0.45245837  0.4288056   0.61432063  0.21840791
  0.21931601  0.49155284  0.08241292  0.19921792  1.17625846  0.91223519
  0.79900308  0.9804085   0.31457154  0.0771426   0.64573555  0.09482758
  0.07860417  0.25249731  0.86631916  0.42039081 -0.23754653  0.12004383
  0.56426226  0.13173728  0.08221643  0.9619988   0.41831297  0.82585984
  0.46029667  0.15507331  0.49194507  0.25046209  0.70059033  0.87505484
  0.79801381  0.82893174  0.14893851  0.82949546  0.93704773  0.34487272
  0.87419564  0.40518786  0.52700755  0.59429094  0.38932606  0.73533801
  0.73429032  0.71888444  0.17436343  0.59793083  0.49738468  0.12726749
  0.65483994  0.92125514  0.90901293  0.49813292  0.92997644  0.56957845
  0.03185094  0.81512059  0.33750717]
2020-11-22 22:00:14,032 - pyswar

preoptim  :  0.17396535093875548
cost =  -13195033.800299179


pyswarms.single.local_best: 100%|██████████|500/500, best_cost=-1.32e+7
2020-11-22 22:02:47,812 - pyswarms.single.local_best - INFO - Optimization finished | best cost: -13192597.251862805, best pos: [ 0.5153883   0.11716764  0.45245837  0.4288056   0.61432063  0.21840791
  0.21931601  0.49155284  0.08241291  0.19921793  1.1762585   0.91223519
  0.79900308  0.9804085   0.31457154  0.0771426   0.64573555  0.09482758
  0.07860417  0.2524973   0.86631916  0.42039081 -0.23754651  0.12004383
  0.56426226  0.13173728  0.08221643  0.9619988   0.41831297  0.82585984
  0.46029667  0.15507331  0.49194507  0.25046209  0.70059033  0.87505484
  0.79801381  0.82893174  0.14893851  0.82949546  0.93704773  0.34487272
  0.87419564  0.40518787  0.52700755  0.59429094  0.38932606  0.73533801
  0.73429032  0.71888444  0.17436343  0.59793083  0.49738468  0.12726749
  0.65483994  0.92125514  0.90901293  0.49813292  0.92997644  0.56957845
  0.03185093  0.81512059  0.33750717]
2020-11-22 22:02:47,824 - pyswar

preoptim  :  0.18266527291356827
cost =  -13192597.251862805


pyswarms.single.local_best: 100%|██████████|500/500, best_cost=-1.32e+7
2020-11-22 22:05:22,486 - pyswarms.single.local_best - INFO - Optimization finished | best cost: -13191564.16576794, best pos: [ 0.5153883   0.11716764  0.45245837  0.4288056   0.61432063  0.21840791
  0.21931601  0.49155284  0.08241291  0.19921793  1.1762585   0.91223519
  0.79900308  0.9804085   0.31457154  0.0771426   0.64573555  0.09482758
  0.07860417  0.2524973   0.86631916  0.42039081 -0.23754651  0.12004383
  0.56426226  0.13173728  0.08221643  0.9619988   0.41831297  0.82585984
  0.46029667  0.15507331  0.49194507  0.25046209  0.70059033  0.87505484
  0.79801381  0.82893174  0.14893851  0.82949546  0.93704773  0.34487272
  0.87419564  0.40518787  0.52700755  0.59429094  0.38932606  0.73533801
  0.73429032  0.71888444  0.17436343  0.59793083  0.49738468  0.12726749
  0.65483994  0.92125514  0.90901293  0.49813292  0.92997644  0.56957845
  0.03185093  0.81512059  0.33750717]
2020-11-22 22:05:22,504 - pyswarm

preoptim  :  0.18266527291356827
cost =  -13191564.16576794


pyswarms.single.local_best:  59%|█████▉    |295/500, best_cost=-1.32e+7


KeyboardInterrupt: 

In [10]:
#========================================================================
# create pdb
#------------------------------------------------------------------------
#Increase structure Size

xyz4pdb = myUtils.convert2xyz(myUtils(), n, bestMat) 
scale=100/np.amax(xyz4pdb)
xyz4pdb = xyz4pdb* scale
#output pdb file
outputData= np.round_(xyz4pdb ,3 )

outputFile = 'yolo.pdb' #output directory.


if os.path.exists(outputFile):
    os.remove(outputFile)

output = outputObj(outputData,outputFile)

pdbMaker = makePDB()
makePDB.mat2pdb(pdbMaker, output) # Converts the mat XYZ coordinates to PDB format.

outputting PDB in file :  yolo.pdb
done! closing file...

